## Aravec Model Word Embedding
* github model source link : https://github.com/bakrianoo/aravec

## Install/Load the required modules

In [1]:
import gensim
import re ##regularexpression
import spacy
import nltk
import util as ut


In [2]:
from gensim import matutils  # utility fnc for pickling, common scipy operations etc
import numpy as np
import math

In [3]:
#model = gensim.models.Word2Vec.load(
 #   "C:/Users/PC/Desktop/ALL/II3IA/NLP/full_grams_cbow_100_twitter.mdl")
#model.wv.save_word2vec_format("./spacyModel/aravec.txt")


In [4]:
nlp = spacy.load("./spacy.aravec.model/")

In [5]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = ut.clean_str(text)
        return self.tokenizer(preprocessed)


In [6]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)


# Text preprocessing

In [37]:
text1 = "طبخ الطباخ المرطبات"
text11 = "طبخ الطباخ المركبات"
text2 = "حقق منتخبنا الوطني الاول لكرة القدم كل اهدافه المطلوبة"
text22 = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"
text3 = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون والاذاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"
text33 = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"
text4 = "منتخبنا يتألق ويهم الشباك"
text44 = "منتخبنا يتألق ويهز الشباك"
text5 ="اصطاد الصياد سمكة"
text55 ="اصطاد الصياد سكة"
text6 ="لكن قد لا يعرف الكثيرون أن موهبتها التمثيلية وجدت طريقها إلى عالم الفن بالصدفة البحته"
text66 = "لكن قد لا يعرف الكثيرون أن موهبتها التمثيلية وجدت طريقها إلى عالم الفن بالصدفة البحته"
text7 = "شهد السباق تفاعلا وحضورا كبيرا من محبي هذه الرياضة الاصيلة كونها تمثل رمزا للاصالة العربية ونهجا على طريق الاجداد"
text77 = "شهد السباق تفاعلا وحضورا كبيرا من محبي هذه الرياضة الاصيلة كونها تمثل رمزا للاصالة العبرية ونهجا على طريق الاجداد"
text8=""
text = text77


## 1) Least Similar vector
* odd word is the one that is the least similar to all other words.


In [48]:
import numpy as np
def min_max_similarity(text):
    origin = nltk.word_tokenize((ut.very_clean(text)))  # preprocessing
    text = nltk.word_tokenize(ut.lemmatize(ut.very_clean(text))) #lemmatizing
    token = [nlp(x) for x in text] #embedding
    avg_list=[]
    for i in range(len(token)): 
        sim_list=[]
        for j in range(len(token)):
            sim_list.append(token[i].similarity(token[j]))
        avg_list.append(np.mean(sim_list))
        #print(token[i], " = ", avg_list[i])
    print("score de chaque mot")
    return(origin[avg_list.index(min(avg_list))])
min_max_similarity(text11)

طبخ  =  0.5630887768772145
طباخ  =  0.5994759343985119
مركب  =  0.45359232925455717
score de chaque mot


'المركبات'

## 2) Furthest Vector
* odd word is the word whose vector is the furthest(considering cosine similarity as distance) from the context vector of the sentence.

In [42]:
#Un mot est considéré erroné si son vecteur est éloigné 
# (par rapport à un seuil) du vecteur moyenne/somme de son contexte 
# context ~ (vecteur moyenne des autres mots)

def context_vec(token, word):
    avg = np.mean([nlp(w).vector for w in token if w != word],
                  axis=0)  # avg euclidean distance
    return avg

#-----------------------
#distance(similarity) between two vectors
def similarity_cosine(vec1, vec2):
    cosine_similarity=0
    cosine_similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
    return cosine_similarity
#---------------------------------------


def odd_word(text):
    origin = nltk.word_tokenize((ut.very_clean(text))) #preprocessing
    text = nltk.word_tokenize(ut.lemmatize(ut.very_clean(text))) #lemmatizing
    sptext = [nlp(x) for x in text] #embedding
    l = list([])
    for word in sptext:
        context = context_vec(sptext, word)
        #l = [math.dist(nlp(w).vector,context) for w in token]
        l.append(similarity_cosine(nlp(word).vector, context))
    farthestword = origin[l.index(min(l))]
    return(farthestword)


odd_word(text11)


'المركبات'

## Mesure de performance